# Import the dataset (embeddings and estimates)

In [ ]:
import pandas as pd

df = pd.read_pickle('df_All_Embeddings.pkl')

print(df['all_embeddings'].shape)
print(df['Estimate'].shape)

# criando datasets X e y (ver se tem label repositorio no df_Req, se não tiver criar) # TODO remover
X = df['all_embeddings'].iloc[:].values
y = df['Estimate'].iloc[:].values
del df

(23313,)
(23313,)


## Continuing from where you stopped

In [ ]:
# Note: This part of the notebook is due to I haved stopped the experiment run before its end,
# and continued from a specific iteration

# If your intent is to run the intire experiment at once, ignore this code frame, and set N to 0
# in the fit function, in the model class, bellow.

partial_results = [{'mae': 3.8671381, 'mse': 80.061966, 'mdae': 1.9161946, 'max_error': 103.7653, 'r2_score': 0.31510945289217973},
{'mae': 3.575684, 'mse': 56.81785, 'mdae': 1.9571185, 'max_error': 96.10432, 'r2_score': 0.37313219625814975},
{'mae': 3.5730035, 'mse': 59.274582, 'mdae': 1.7877096, 'max_error': 88.33195, 'r2_score': 0.4130561554123663},
{'mae': 3.4315414, 'mse': 55.22547, 'mdae': 1.8260198, 'max_error': 88.686386, 'r2_score': 0.38604373521737634},
{'mae': 3.4607165, 'mse': 54.236168, 'mdae': 1.9114883, 'max_error': 94.3574, 'r2_score': 0.4353173485532321},
{'mae': 3.912162, 'mse': 73.2315, 'mdae': 1.9148998, 'max_error': 93.76645, 'r2_score': 0.353372377238224}
]
acc_vector = {
      'mae': [],
      'mse': [],
      'mdae': [],
      'max_error': [],
      'r2_score': [],
  }
for partial_i in partial_results:
  for key in partial_i.keys():
    acc_vector[key].append(partial_i[key])

print('resuming from:',acc_vector,'-----------------------------\n',sep='\n')


resuming from:
{'mae': [3.8671381, 3.575684, 3.5730035, 3.4315414, 3.4607165, 3.912162], 'mse': [80.061966, 56.81785, 59.274582, 55.22547, 54.236168, 73.2315], 'mdae': [1.9161946, 1.9571185, 1.7877096, 1.8260198, 1.9114883, 1.9148998], 'max_error': [103.7653, 96.10432, 88.33195, 88.686386, 94.3574, 93.76645], 'r2_score': [0.31510945289217973, 0.37313219625814975, 0.4130561554123663, 0.38604373521737634, 0.4353173485532321, 0.353372377238224]}
-----------------------------


In [ ]:
import keras
from keras.layers import Input, Dense
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, median_absolute_error, r2_score, max_error
from keras.callbacks import EarlyStopping
from sklearn.base import BaseEstimator

# from joblib import Parallel
from tqdm import tqdm
import gc
import numpy.ma as ma
import math

def desNaNify(x):
  if math.isnan(x):
    return 9999 # retorna um número grande ao invés de nan, para não dar erro no mse
  return x

class MyEstimator(BaseEstimator):
  model=None
  
  def __init__(self, n1 = 256, n2 = 256, n3 = 256, lr = 0.001, batch_size = 256, beta_1 = 0.99, beta_2 = 0.999):
    self.beta_1=beta_1
    self.beta_2=beta_2
    self.n1=n1
    self.n2=n2
    self.n3=n3
    self.lr=lr
    self.batch_size=batch_size

  def fit(self, X_train, y, X_valid, y_valid):
    """Trains a model with the given parameters and given training set.
    
    Returns the trained model, embedded with its results.
    """
    self.input_layer = Input(shape=(2048,))
    # model = expand_dims(input_layer,axis=-1)
    self.model = Dense(self.n1, kernel_initializer='normal', activation='relu')(self.input_layer) #(model)# Híperparâmetros [LSTM]: dropout, recurrent_dropout, número de nós
    self.model = Dense(self.n2, kernel_initializer='normal', activation='relu')(self.model) # Híperparâmetros [DENSE]: Número de nós por camada
    self.model = Dense(self.n3, kernel_initializer='normal', activation='relu')(self.model)
    self.model = Dense(1, kernel_initializer='normal', activation='linear')(self.model)
    self.model = keras.Model(inputs = self.input_layer, outputs = self.model)

    adam = Adam(learning_rate = self.lr, beta_1=self.beta_1, beta_2=self.beta_2, epsilon = None, decay=0.01, amsgrad = False)
    # Híperparâmetros [Adam]: lr, beta1, beta2, epsilon, decay

    self.model.compile(loss = 'mse', optimizer= adam, metrics=['mae'],steps_per_execution=1,)

    earlystopping = EarlyStopping(monitor='val_mae', mode='auto', verbose=0, patience=10, restore_best_weights=True)

    # print(self.model.summary())
    self.result = self.model.fit(
                    X_train,
                    y,
                    batch_size=self.batch_size,
                    epochs=48,
                    callbacks=[earlystopping],
                    validation_data=(X_valid, y_valid),
                    verbose=0
                  )
    del adam
    del earlystopping
    return self

  def predict(self, X):
    """Returns y_hat"""
    return self.model.predict(X) # y_hat

def product_dict(**kwargs):
    """Make the product of the settings therms."""
    import itertools
    keys = kwargs.keys()
    vals = kwargs.values()
    for instance in itertools.product(*vals):
        yield dict(zip(keys, instance))

def my_iterable(config:dict, random_seed:int=None):
    """Generates and returns a sorted list of settings pairs, sorted randomly (with seed)"""
    import random

    if random_seed != None: random.seed(random_seed)

    aux = list(product_dict(**config))
    random.shuffle(aux)
    return aux

def my_accuracy(y_test,y_hat):
  """Returns a set of evaluation metrics of the output y_hat, in comparison to the real values y_test."""
  return {
      'mae': mean_absolute_error(y_test,y_hat),
      'mse': mean_squared_error(y_test,y_hat),
      'mdae': median_absolute_error(y_test,y_hat),
      'max_error': max_error(y_test,y_hat),
      'r2_score': r2_score(y_test,y_hat)
  }

def nested(X,y,config={},n_iter:int=None,k:int=10):
  N= 6
  thetamin_vector = []
  acc_vector = { # Note: you can comment this declaration if continuing from a partial result set
      'mae': [],
      'mse': [],
      'mdae': [],
      'max_error': [],
      'r2_score': [],
  }
  # outer cross validation (k-fold)
  cv_outer = KFold(n_splits=k, shuffle=True, random_state=1)
  i=0
  iterable_parameters = my_iterable(config=config,random_seed=123)[:n_iter]
  if(n_iter!=None and n_iter == len(iterable_parameters)):
    print('\n',(k-N)*(n_iter*k+1),' models will be fitted\n',sep='')
  else:
    print('\n',(k-N)*(len(iterable_parameters)*k+1),' models will be fitted\n',sep='')
  print('------------------------------------------------\n')

  for tr_o, te_o in cv_outer.split(X):
    i+=1
    if i>N: # N is the last iteration the code successfully reported results
      print('\n[STARTED] OUTER_CV ',i,'/',k,sep='')
      X_train_o, X_test_o = X[tr_o], X[te_o]
      y_train_o, y_test_o = y[tr_o], y[te_o]
      accmin=0.0
      thetamin=None
      j=0
      for theta in iterable_parameters: #grid/random/hyperopt search
        j+=1
        print('\nCurrent parameter configuration being tested [',j,'/',len(iterable_parameters),']:',sep='')
        print(theta,'\n',sep='')
        acc=0.0
        # inner cross validation (k-fold)
        cv_inner=KFold(n_splits=k, shuffle=True, random_state=2)
        l=0
        for tr_i, te_i in tqdm(cv_inner.split(X_train_o)):
          X_train_i, X_test_i = X_train_o[tr_i], X_train_o[te_i]
          y_train_i, y_test_i = y_train_o[tr_i], y_train_o[te_i]

          fit_params = {'X_valid': X_test_i, 'y_valid': y_test_i}
          model_i=MyEstimator(**theta).fit(X_train_i,y_train_i,**fit_params) #fit
          y_hat = model_i.predict(X_test_i)
          y_hat = np.vectorize(lambda x: desNaNify(x))(y_hat)
          print('\n',y_hat[ ma.masked_invalid(y_hat)._mask ],' ; ',X_test_i[ ma.masked_invalid(X_test_i)._mask ],sep='',end='')
          # evaluate the model
          mse = mean_squared_error(y_test_i,y_hat)
          acc+=mse
          l+=1
          print('\nINNER_CV ',l,'/',k,' concluded',sep='')
          del fit_params
          del model_i
          del y_hat
          del mse
          del X_train_i, X_test_i, y_train_i, y_test_i
          del tr_i, te_i
        # END OF INNER LOOP
        print('HP RESULTS [',j,'/',len(iterable_parameters),']: ',acc,sep='')
        if acc < accmin or j==1:
          accmin = acc
          thetamin = theta
        del acc
        del theta
        del cv_inner
        gc.collect()
      # END OF RANDOMSEARCH LOOP
      fit_params={'X_valid': X_test_o, 'y_valid': y_test_o}
      model_o = MyEstimator(**thetamin).fit(X_train_o,y_train_o,**fit_params)
      y_hat = model_o.predict(X_test_o)
      aux=my_accuracy(y_test_o,y_hat)
      for key in acc_vector.keys():
        acc_vector[key].append(aux[key])
      thetamin_vector.append(thetamin)
      print('\n[FINISHED] OUTER_CV ',i,'/',k,sep='')
      print('Iteration Results:')
      print(thetamin)
      print(aux)
      print('---------------------------------------')
      del aux
      del thetamin
      del y_hat
      del model_o
      del fit_params
      del X_train_o, X_test_o, y_test_o, y_train_o
  # END OF OUTER LOOP
  accfinal=dict()
  for key in acc_vector.keys():
    accfinal['mean_'+key]=np.mean(acc_vector[key])
    if key in ['r2_score','mae','mse']:
      accfinal['std_'+key]=np.std(acc_vector[key])
  accfinal['train_results']=acc_vector
  accfinal['thetas']=thetamin_vector
  del acc_vector
  del thetamin_vector
  return accfinal
# END OF NESTED FUNCTION

X = np.asarray([np.asarray(x).astype('float32') for x in X])
y = np.asarray(y).astype('float32')
config = { # 2*5*2*9=180
        'n1': [512], #2
        'n2': [512], #1
        'n3': [512], #1
        "lr": [1e-5,1e-4,1e-3,1e-2], #5
        "batch_size": [64,128], #2
        "beta_1": [0.9,0.95,0.99], #3
        "beta_2": [0.9,0.99,0.999], #3
      }
results = nested(X,y,
                 config=config,
                 n_iter=100,
                 k=10)
print('\n------------------------------------------\nNESTED CV RESULTS:',results,sep='\n')

Successfully created Issue Training status 0

2884 models will be fitted

------------------------------------------------


[STARTED] OUTER_CV 7/10

Current parameter configuration being tested [1/72]:
{'n1': 512, 'n2': 512, 'n3': 512, 'lr': 0.01, 'batch_size': 64, 'beta_1': 0.99, 'beta_2': 0.9}



0it [00:00, ?it/s]

66/66 [==============================] - 1s 6ms/step


1it [01:24, 84.89s/it]


[] ; []
INNER_CV 1/10 concluded
66/66 [==============================] - 0s 7ms/step

[] ; []

2it [02:36, 76.95s/it]


INNER_CV 2/10 concluded


2it [02:41, 80.69s/it]


KeyboardInterrupt: ignored

# Saving the results to a file

In [ ]:
import pickle
from datetime import datetime

now = datetime.now()
current_time = now.strftime("%d%m%y-%H%M")

with open(f'results-{current_time}.pickle', 'wb') as handle:
    pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)
print('\nThe results were been written to a results.pckl file!')

# Commands to open results file

In [ ]:
# !pip3 install pickle5 --quiet
# import pickle5 as pickle
# with open('results-ddmmyyyy-HHMM.pickle', 'rb') as handle:
#   unserialized_data = pickle.load(handle)
# print(unserialized_data['thetas'][0])


{'n1': 256}
